In [1]:
import pandas as pd

In [2]:
movies_df = pd.read_json('https://raw.githubusercontent.com/vega/vega-datasets/master/data/movies.json')
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3201 entries, 0 to 3200
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Title                   3200 non-null   object 
 1   US Gross                3194 non-null   float64
 2   Worldwide Gross         3194 non-null   float64
 3   US DVD Sales            564 non-null    float64
 4   Production Budget       3200 non-null   float64
 5   Release Date            3201 non-null   object 
 6   MPAA Rating             2596 non-null   object 
 7   Running Time min        1209 non-null   float64
 8   Distributor             2969 non-null   object 
 9   Source                  2836 non-null   object 
 10  Major Genre             2926 non-null   object 
 11  Creative Type           2755 non-null   object 
 12  Director                1870 non-null   object 
 13  Rotten Tomatoes Rating  2321 non-null   float64
 14  IMDB Rating             2988 non-null   

In [3]:
# Add a column of the release year
movies_df['Year'] = pd.to_datetime(movies_df['Release Date']).dt.year
# A list columns that will appear in the final data frame
movies_df['US_Revenue'] = (movies_df['US Gross'])/1000_000
column_list = ['Title', 'Major Genre', 'Running Time min', 'Year', 'US_Revenue', 'IMDB Rating', 'MPAA Rating']

# Filter out lines with errors or NaNs in 'Running_Time_min', 'Year', 'Major_Genre' columns
movies_df = (movies_df.query('Year > 1980')
                     .dropna(subset = column_list)
                     .loc[:, column_list]
                     .reset_index()
                     .drop(columns = ['index']))

In [4]:
movies_df.shape

(1134, 7)

In [5]:
movies_df.describe()

,Running Time min,Year,US_Revenue,IMDB Rating
count,1134.000000,1134.000000,1134.000000,1134.000000
mean,110.354497,2002.584656,71.652644,6.308907
std,20.166025,4.232786,70.635363,1.145874
min,46.000000,1985.000000,0.012604,1.700000
25%,95.000000,1999.000000,27.685357,5.600000
50%,107.500000,2002.000000,49.605446,6.400000
75%,121.000000,2006.000000,91.818142,7.100000
max,222.000000,2039.000000,600.788188,9.100000


In [6]:
movies_df.to_csv('../data/clean/movies_clean_df.csv')